In [2]:
from owlready2 import *
onto_path.append("/home/alice/")
onto = get_ontology("http://www.semanticweb.org/alice/ontologies/2023/3/ontology_new#").load()

In [3]:
with onto:
    class Predicates(Thing):
        pass
    class Functions(Thing):
        pass
    class Actions(Thing):
        pass
    class Types(Thing):
        pass
    class Objects(Thing):
        pass

AllDisjoint([Predicates,Functions,Actions,Types,Objects])

AllDisjoint([ontology_new.Predicates, ontology_new.Functions, ontology_new.Actions, ontology_new.Types, ontology_new.Objects])

In [4]:
list(onto.classes())

[ontology_new.Actions,
 ontology_new.Predicates,
 ontology_new.Functions,
 ontology_new.Objects,
 ontology_new.Types]

In [13]:
#with onto:
 #sync_reasoner()

In [5]:
with onto:

    class has_effect_predicates(ObjectProperty):
        domain    = [Actions]
        range     = [Predicates]

    class is_effect_predicates(ObjectProperty):
        domain    = [Predicates]
        range     = [Actions]
        inverse_property = has_effect_predicates

    class has_effect_function(ObjectProperty):
        domain    = [Actions]
        range     = [Functions]

    class is_effect_function(ObjectProperty):
        domain    = [Functions]
        range     = [Actions]
        inverse_property = has_effect_function

    class has_object(ObjectProperty):
        domain    = [Predicates]
        range     = [Objects]

    class is_object(ObjectProperty):
        domain    = [Objects]
        range     = [Predicates]
        inverse_property = has_object

    class has_type(ObjectProperty):
        domain    = [Objects]
        range     = [Types]

    class is_type(ObjectProperty):
        domain    = [Types]
        range     = [Objects]
        inverse_property = has_type

In [7]:
with onto:
    class has_value(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Functions]
        range = [float]
    class isGrounded(DataProperty, FunctionalProperty): #functional means that each one has a single value
        domain =[Predicates]
        range = [float]
    class hasOperator(DataProperty): #is not a functional, each predicate can have more than one operator
        #domain =[Predicates]
        range = [float]

check inverse
a = Actions("go")
p = Predicates("at")
#a.has_effect_predicates.append(p)
p.is_effect_predicates=[a]
a.has_effect_predicates
p.is_effect_predicates
is_effect_predicates.domain

In [8]:
domain_path = "/home/alice/PROPER_LPG/"
if os.path.isfile(domain_path+'/prova_domain.pddl'):
    output_path=domain_path+'/prova_domain.pddl'
    with open(output_path, "r") as domain_file:
        raw_domain = domain_file.readlines()


In [9]:
last_action=""
actions=[]
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
        actions.append(last_action)
actions.pop(0)
actions_objects={}
for a in actions:
    actions_objects[a]=Actions(a)

In [11]:
#FIND TYPES AND ADD TO THE ONTOLOGY
types=[]
t=False
for p in raw_domain:
        if (")" in p):
            t=False
        if t==True:
            types.append(p.replace("\t","").replace("\n",""))
        if (":types" in p):
            t=True
types_objects={}
for t in types:
   types_objects[t]=Types(t)

In [12]:
#PREDICATES
predicates=[]
t=False
for p in raw_domain:
        if (":action" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")].replace(" ","")
                
            else:
                new=p[p.find("(")+1:p.find(")")].replace(" ","")
                
            if new!="":   
                predicates.append(new)
        if (":predicates" in p):
            t=True
predicates_objects={}
for t in predicates:
    predicates_objects[t]=Predicates(t)


In [13]:
predicates_objects

{'production_room': ontology_new.production_room,
 'assembly_room': ontology_new.assembly_room,
 'presented_task': ontology_new.presented_task,
 'at': ontology_new.at,
 'human_present': ontology_new.human_present,
 'block_to_deliver': ontology_new.block_to_deliver,
 'empty_robot': ontology_new.empty_robot,
 'finished': ontology_new.finished,
 'extro': ontology_new.extro,
 'intro': ontology_new.intro,
 'computed_e': ontology_new.computed_e,
 'consc': ontology_new.consc,
 'unsc': ontology_new.unsc,
 'computed_c': ontology_new.computed_c,
 'agree': ontology_new.agree,
 'disagree': ontology_new.disagree,
 'computed_a': ontology_new.computed_a}

In [14]:
functions=[]
t=False
for p in raw_domain:
        if (":predicates" in p):
            t=False
        if t==True:
            if "?" in p:
                new=p[p.find("(")+1:p.find("?")]
                
            else:
                new=p[p.find("(")+1:p.find(")")]
                
            if new!="":   
               functions.append(new)
        if (":functions" in p):
            t=True
function_objects={}
for t in functions:
    function_objects[t]=Functions(t)

In [39]:
#connect all the relations between actions predicates and functions
t=False
last_action=""
for p in raw_domain:
    if (":durative-action" in p) or (":action" in p):
        t=False
        if (":durative-action" in p):
            last_action=p.replace(":durative-action","").replace("( ","").replace("\n","")
        elif (":action" in p):
            last_action=p.replace(":action","").replace("( ","").replace("\n","")
    if t==True:
        if ";" not in p:
            operation=p.replace("and","").replace("at end","").replace("("," ").replace(")"," ")
            pred=operation.replace("decrease","").replace("increase","").replace("assign","").replace("not","").split()
            #operation=operation.split()
           
            if pred==[]:
                continue
            else:
                   
               check=str(pred[0])
               for l in predicates:
                   if check==l:
                       actions_objects[last_action].has_effect_predicates.append(predicates_objects[check])
                       break
                    
               for l in functions:
                   if check==l:
                       actions_objects[last_action].has_effect_function.append(function_objects[check])
                       break

    if ":effect" in p:
        t=True

<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class 'str'>
<class

In [25]:
#prendo un'azione
#prendo tutti gli effetti
#se un effetto è predicato metto has_predicate_effect
#se è una funzione metto come has_function_effect

In [17]:
for i in Functions.instances():
    print(i)

ontology_new.no_of_box
ontology_new.max_no_blocks
ontology_new.no_blocks
ontology_new.dur
ontology_new.extroversion_coefficient
ontology_new.desired_interaction
ontology_new.interaction_level
ontology_new.conscientious_coefficient
ontology_new.desired_scrupulousness
ontology_new.scrupulousness_level
ontology_new.agreeableness_coefficient
ontology_new.desired_agreeableness
ontology_new.agreeableness_level
ontology_new.baseline


TypeError: 'NoneType' object is not subscriptable

In [77]:
onto.save()

In [14]:
predicates

['production_room ',
 'assembly_room ',
 'presented_task ',
 'at ',
 'human_present',
 'block_to_deliver',
 'empty_robot',
 'finished',
 'extro',
 'intro',
 'computed_e',
 'consc',
 'unsc',
 'computed_c',
 'agree',
 'disagree',
 'computed_a']

In [ ]:
#p=Predicates("finished")
#destroy_entity(p)

data properties
        int
        float
        bool
        str (string)
        owlready2.normstr (normalized string, a single-line string)
        owlready2.locstr (localized string, a string with a language associated)
        datetime.date
        datetime.time
        datetime.datetime

In [ ]:
f = Functions("no_of_box")
f.has_value=[5]

In [ ]:
f.has_value

In [ ]:
f.has_value=[2]